# **Proyecto EDA - Exploratory Data Analysis**

## Año 2020

In [1]:
2+2

4

In [2]:
import numpy as np
import pandas as pd
pd.options.mode.copy_on_write = True
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
direccion1="./Datasets/2020_Accidentalidad.csv"
direccion2="./Datasets/2021_Accidentalidad.csv"
direccion3="./Datasets/2022_Accidentalidad.csv"
direccion4="./Datasets/2023_Accidentalidad.csv"
direccion5="./Datasets/2024_Accidentalidad.csv"
direccion6="./Datasets/2025_Accidentalidad.csv"

In [5]:
df_accidentes_2020=pd.read_csv(direccion1, sep=";", encoding="utf-8")
df_20=df_accidentes_2020.copy()

In [6]:
#En primer lugar, fechamos un vistazo a las 5 primeras filas
df_20.head(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga
0,2019S040008,07/09/2020,23:00:00,"CALL. SAN MAXIMILIANO, 38",38,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,444578,4475148,N,NaN
1,2019S040008,07/09/2020,23:00:00,"CALL. SAN MAXIMILIANO, 38",38,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo,Despejado,VMU eléctrico,Conductor,De 25 a 29 años,Mujer,NaN,NaN,444578,4475148,N,NaN
2,2020S000001,01/01/2020,1:15:00,AVDA. CANILLEJAS A VICALVARO / CALL. SILFIDE,1,20.0,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,NaN,Turismo,Conductor,De 18 a 20 años,Hombre,NaN,NaN,447894,4476691,N,NaN
3,2020S000001,01/01/2020,1:15:00,AVDA. CANILLEJAS A VICALVARO / CALL. SILFIDE,1,20.0,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,NaN,Turismo,Conductor,De 30 a 34 años,Hombre,NaN,NaN,447894,4476691,S,NaN
4,2020S000002,01/01/2020,1:20:00,"CALL. SILVANO, 31",31,16.0,HORTALEZA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,445094,4478370,N,NaN


In [7]:
#Visualizamos información general del dataframe:
print(f"La base de datos tiene {len(df_20.columns)} campos y {len(df_20)} registros.")
df_20.info()

La base de datos tiene 19 campos y 32433 registros.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32433 entries, 0 to 32432
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   num_expediente        32433 non-null  object 
 1   fecha                 32433 non-null  object 
 2   hora                  32433 non-null  object 
 3   localizacion          32433 non-null  object 
 4   numero                32431 non-null  object 
 5   cod_distrito          32431 non-null  float64
 6   distrito              32431 non-null  object 
 7   tipo_accidente        32433 non-null  object 
 8   estado_meteorológico  29237 non-null  object 
 9   tipo_vehiculo         32296 non-null  object 
 10  tipo_persona          32433 non-null  object 
 11  rango_edad            32433 non-null  object 
 12  sexo                  32433 non-null  object 
 13  cod_lesividad         17631 non-null  float64
 14  lesividad         

In [8]:
#Visualizamos información de las variables numéricas
#Variables numéricas: cod_lesividad, coordenada_x_utm, coordenada_y_utm, positiva_alcohol
df_20.describe()

,cod_distrito,cod_lesividad,positiva_droga
count,32431.000000,17631.000000,82.0
mean,10.074034,9.862004,1.0
std,5.619433,4.789128,0.0
min,1.000000,1.000000,1.0
25%,5.000000,7.000000,1.0
50%,10.000000,14.000000,1.0
75%,15.000000,14.000000,1.0
max,21.000000,77.000000,1.0


In [9]:
#Obtenemos número de nulos y cardinalidad de todas las variables
#Creamos el df de tipificación
def nulos_cardinalidad(df,umbral_categoria, umbral_continua):
    nulls=df.isna().sum().sort_values(ascending=False)
    nulls_rel=(df.isna().sum()/len(df)*100).sort_values(ascending=False)
    card=df.nunique()
    card_rel=df.nunique()/len(df)*100
    tipo=df.dtypes
    df_tipificacion= pd.DataFrame([nulls, nulls_rel, card, card_rel, tipo]).T.rename(columns = {0: "Nulls", 1: "Nulls_rel", 2: "Card", 3: "Card_rel", 4: "Tipo"}) #hace falta poner corchetes
    df_tipificacion["Clasificada_como"]="sin categoría" #partimos asumiendo que todas son categorica
    df_tipificacion.loc[df_tipificacion.Card == 2, "Clasificada_como"] = "Binaria"
    df_tipificacion.loc[df_tipificacion.Tipo == "datetime64[ns]", "Clasificada_como"] = "Fecha"
    df_tipificacion.loc[(df_tipificacion.Card > 2) & (df_tipificacion.Card < umbral_categoria), "Clasificada_como"] = "Categórica"
    df_tipificacion.loc[(df_tipificacion.Tipo == "float64") & (df_tipificacion.Card > 2) & (df_tipificacion.Card < umbral_categoria), "Clasificada_como"] = "Numérica ordinal"
    df_tipificacion.loc[(df_tipificacion.Tipo == "float64") & (df_tipificacion.Card > umbral_categoria) & (df_tipificacion.Card_rel < umbral_continua), "Clasificada_como"] = "Numérica discreta"
    df_tipificacion.loc[(df_tipificacion.Tipo == "float64") & (df_tipificacion.Card_rel > umbral_continua), "Clasificada_como"] = "Numérica continua"
    return df_tipificacion

In [14]:
print(f"EL dataframe tiene {len(df_20)} registros.")
df_tipificacion=nulos_cardinalidad(df_20,22,30)
df_tipificacion

EL dataframe tiene 32433 registros.


,Nulls,Nulls_rel,Card,Card_rel,Tipo,Clasificada_como
positiva_droga,32351,99.747171,1,0.003083,float64,sin categoría
cod_lesividad,14802,45.638701,9,0.02775,float64,Numérica ordinal
lesividad,14802,45.638701,9,0.02775,object,Categórica
estado_meteorológico,3196,9.854161,7,0.021583,object,Categórica
tipo_vehiculo,137,0.422409,35,0.107915,object,sin categoría
positiva_alcohol,102,0.314494,2,0.006167,object,Binaria
distrito,2,0.006167,21,0.064749,object,Categórica
cod_distrito,2,0.006167,21,0.064749,float64,Numérica ordinal
coordenada_y_utm,2,0.006167,8911,27.475103,object,sin categoría
coordenada_x_utm,2,0.006167,8400,25.899547,object,sin categoría


In [11]:
#Visualizamos los valores de las variables categóricas:
variables_categoricas=df_tipificacion.loc[df_tipificacion.Clasificada_como == "Categórica"].index.to_list()
variables_categoricas

['lesividad',
 'estado_meteorológico',
 'distrito',
 'tipo_persona',
 'tipo_accidente',
 'rango_edad',
 'sexo']

In [12]:
for categoria in variables_categoricas:
    frec_abs=df_20[categoria].value_counts(dropna=False)
    frec_rel=df_20[categoria].value_counts(dropna=False, normalize=True)*100
    df_frec=pd.concat([frec_abs, frec_rel], axis=1)
    df_frec.columns=["Frec. Absoluta", "Frec. Relativa (%)"]

    print(f"Resumen de {categoria}:")
    print(df_frec)
    print(f"Su cardinalidad es: {df_tipificacion.loc[categoria, "Card"]}")
    print("\n"*2)

Resumen de lesividad:
                                                    Frec. Absoluta  \
lesividad                                                            
NaN                                                          14802   
Sin asistencia sanitaria                                      9376   
Asistencia sanitaria sólo en el lugar del accid...            4346   
Ingreso inferior o igual a 24 horas                           1310   
Atención en urgencias sin posterior ingreso                    904   
Asistencia sanitaria inmediata en centro de sal...             826   
Asistencia sanitaria ambulatoria con posterioridad             424   
Ingreso superior a 24 horas                                    408   
Fallecido 24 horas                                              34   
Se desconoce                                                     3   

                                                    Frec. Relativa (%)  
lesividad                                                       

In [13]:
#Convertimos los campos de fecha y hora a datetime y creamos una nueva columna:

df_20["fecha"] = pd.to_datetime(df_20["fecha"], errors="coerce", dayfirst=True) 
df_20["hora"] = pd.to_datetime(df_20["hora"], format="%H:%M:%S", errors="coerce").dt.time 
df_20["fecha_hora"] = df_20.apply( 
                            lambda row: pd.Timestamp.combine(row["fecha"], row["hora"])
                            if pd.notnull(row["fecha"]) and pd.notnull(row["hora"])
                            else pd.NaT,
                            axis=1 )

In [15]:
#Eliminamos dobles espacios y unificamos el formato de los campos: "num_expediente", "localizacion", "numero"
cols_string = ["num_expediente", "localizacion", "numero"] 
for col in cols_string: 
    df_20[col] = df_20[col].astype(str).str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

In [16]:
df_20.head(5)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,tipo_persona,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fecha_hora
0,2019s040008,2020-09-07,23:00:00,"call. san maximiliano, 38",38,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,444578,4475148,N,NaN,2020-09-07 23:00:00
1,2019s040008,2020-09-07,23:00:00,"call. san maximiliano, 38",38,15.0,CIUDAD LINEAL,Choque contra obstáculo fijo,Despejado,VMU eléctrico,Conductor,De 25 a 29 años,Mujer,NaN,NaN,444578,4475148,N,NaN,2020-09-07 23:00:00
2,2020s000001,2020-01-01,01:15:00,avda. canillejas a vicalvaro / call. silfide,1,20.0,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,NaN,Turismo,Conductor,De 18 a 20 años,Hombre,NaN,NaN,447894,4476691,N,NaN,2020-01-01 01:15:00
3,2020s000001,2020-01-01,01:15:00,avda. canillejas a vicalvaro / call. silfide,1,20.0,SAN BLAS-CANILLEJAS,Colisión fronto-lateral,NaN,Turismo,Conductor,De 30 a 34 años,Hombre,NaN,NaN,447894,4476691,S,NaN,2020-01-01 01:15:00
4,2020s000002,2020-01-01,01:20:00,"call. silvano, 31",31,16.0,HORTALEZA,Choque contra obstáculo fijo,Despejado,Turismo,Conductor,De 21 a 24 años,Hombre,NaN,NaN,445094,4478370,N,NaN,2020-01-01 01:20:00


In [37]:
df_20.isna().sum().sort_values(ascending=False)

num_expediente          0
fecha                   0
hora                    0
localizacion            0
numero                  0
cod_distrito            0
distrito                0
tipo_accidente          0
estado_meteorológico    0
tipo_vehiculo           0
tipo_persona            0
rango_edad              0
sexo                    0
cod_lesividad           0
lesividad               0
coordenada_x_utm        0
coordenada_y_utm        0
positiva_alcohol        0
positiva_droga          0
fecha_hora              0
dtype: int64

In [ ]:
#TRATAMOS LOS NULOS

# 1. Estado meteorlogico
df_20['estado_meteorológico'] = df_20['estado_meteorológico'].fillna('No consta') 
# 2. TIPO VEHÍCULO 
mask_no_aplica = df_20['tipo_persona'].str.lower().isin(['peatón', 'testigo']) 
# Peatón y testigo → No aplica 
df_20.loc[ mask_no_aplica & df_20['tipo_vehiculo'].isna(), 'tipo_vehiculo' ] = 'No aplica' 
# El resto de nulos → No consta 
df_20['tipo_vehiculo'] = df_20['tipo_vehiculo'].fillna('No consta') 
# # 3. LESIVIDAD (ORDINAL) 
df_20['cod_lesividad'] = df_20['cod_lesividad'].fillna(14) 
df_20['lesividad'] = df_20['lesividad'].fillna('Sin asistencia sanitaria') 
# # 4. ALCOHOL Y DROGAS 
df_20['positiva_alcohol'] = df_20['positiva_alcohol'].fillna('No consta') 
df_20['positiva_droga'] = df_20['positiva_droga'].fillna('No consta') 
# #5. Quedan 8 nulos, los eliminamos (32433 ---> 32429 registros)
df_20=df_20.dropna()